## <p style="background-color:#9d4f8c; font-family:newtimeroman; color:#FFF9ED; font-size:175%; text-align:center; border-radius:10px 10px;">Web Scraping from autoscout.de</p>

<p align="center">
  <img src="https://championstream.de/wp-content/uploads/2022/01/Cars-on-the-Road-Disney-Plus.jpg" width=1300/>
</p>

<style>
.my-anchor2 {
  background-color: #9d4f8c;
  font-family: newtimeroman;
  color: White;
  font-size: 100%;
  text-align: center;
  border-radius: 10px;
}
</style>
## <a class="my-anchor2" id="content">  Content  </a>

* [Ziel](#ziel)
* [Import Bibliotheken](#imp_bib)
* [Vorbereitung für Data Scraping](#vds)
    * [Beliebteste Marken](#bm)
    * [Lieblingsmodelle beliebter Marken](#lbm)
    * [Informationen, die von der Website abgerufen werden sollen](#info)
* [Erstellung der Daten](#ed)

## <a class="my-anchor2" id="ziel"> Ziel </a>

#### Mein Ziel in diesem Projekt ist es, einen Datensatz aus den auf der Autoscout-Website verkauften Fahrzeugen zu erstellen.

## <a class="my-anchor2" id="imp_bib"> Import Bibliotheken </a>

In [1]:
import warnings as wr
wr.filterwarnings("ignore")

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime
import string
import time

## <a class="my-anchor2" id="vds">Vorbereitung für Data Scraping</a>

* Zunächst werden wir versuchen herauszufinden, <Br>welche Informationen wir unter welchen Klassen erhalten können.

## <a class="my-anchor2" id="bm"> Beliebteste Marken </a>

In [2]:
# Wenn Sie von Google auf die Website gehen, sehen wir, dass es eine Liste als die am meisten bevorzugten Marken auf dem Bildschirm, die Sie begrüßt.
# Scrapen wir diese Marken von der Website

response = requests.get("https://www.autoscout24.de/lst?atype=C&cy=D&desc=0&fregfrom=2020&ocs_listing=include&sort=standard&ustate=N%2CU")
soup = bs(response.content, 'html.parser')
liste = soup.find("div", class_="ListHeader_chips__D5d4i")
marke = []
for i in liste:
    if i.text[:-1] != "Top Marken":
        marke.append(i.text[:-1])

marke

['Mercedes-Benz', 'BMW', 'Volkswagen', 'Audi', 'Ford', 'Opel', 'Skoda']

## <a class="my-anchor2" id="lbm"> Lieblingsmodelle beliebter Marken </a>

In [3]:
# Let's take the most preferred models of these brands
models = {} 

for m in marke:
    url = f"https://www.autoscout24.de/lst/{m}?atype=C&cy=D&damaged_listing=&desc=0&fregfrom=2020&ocs_listing=include&powertype=kw&search_id=2766kwgxaib&sort=standard&ustate=N%2CU"
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    liste = soup.find("div", class_="ListHeader_chips__D5d4i")
    models[m] = []  # her marka adı için boş bir liste oluşturun
    for i in liste:
        if i.text[:-1] != "Top Modelle":
            models[m].append(i.text[:-1])  # ilgili marka adının listesine modeli ekleyin

models


{'Mercedes-Benz': ['Vito', 'C 220', 'Sprinter', 'E 220'],
 'BMW': ['X1', 'X3', 'X5', '320', '530'],
 'Volkswagen': ['Golf',
  'Tiguan',
  'T-Roc',
  'Caddy',
  'Touran',
  'Polo',
  'Passat Variant'],
 'Audi': ['A4', 'A3', 'A6', 'A5', 'Q5', 'Q3', 'A1'],
 'Ford': ['Focus', 'Mustang', 'Kuga', 'Fiesta', 'Mondeo', 'S-Max', 'Ranger'],
 'Opel': ['Astra',
  'Corsa',
  'Insignia',
  'Mokka',
  'Adam',
  'Zafira',
  'Grandland X'],
 'Skoda': ['Octavia', 'Fabia', 'Kodiaq', 'Karoq', 'Superb', 'Kamiq', 'Yeti']}

* Sie werden auf der Website als die meistverkauften Fahrzeuge aufgeführt, aber wir wollen keine Kleinbusse auf unserer Liste haben. Wir werden unseren Datensatz hauptsächlich auf Autos aufbauen.

## <a class="my-anchor2" id="info"> Informationen, die von der Website abgerufen werden sollen </a>

In [4]:
# Schauen wir uns nun an, welche Daten wir in unsere Daten aufnehmen können.
# Wir können auf die Preisinformationen, die Preisbewertung des Produkts und einige Informationen wie km, ps des Fahrzeugs zugreifen.

features = []
url = f"https://www.autoscout24.de/lst/volkswagen/golf/re_2015?source=listpage_seo_links"
response = requests.get(url)
soup = bs(response.content, 'html.parser')
liste = soup.find_all("div", class_="ListItem_wrapper__J_a_C")
df_list = []
for box in liste:
    if box is not None:
        price_elem = box.find("p", class_="Price_price__WZayw PriceAndSeals_current_price__XscDn")
        lb_elem = box.find("div", class_="scr-price-label PriceAndSeals_price_info___KqSg")
        if price_elem is not None and lb_elem is not None:
            pr = price_elem.text
            lb = lb_elem.text
            features.append({'Price': pr, 'Label': lb})
features


[{'Price': '€ 13.750,-', 'Label': 'Sehr guter Preis'},
 {'Price': '€ 12.830,-', 'Label': 'Sehr guter Preis'},
 {'Price': '€ 20.990,-', 'Label': 'Keine Angabe'},
 {'Price': '€ 28.390,-', 'Label': 'Guter Preis'},
 {'Price': '€ 13.999,-', 'Label': 'Guter Preis'},
 {'Price': '€ 21.990,-', 'Label': 'Fairer Preis'},
 {'Price': '€ 20.379,-', 'Label': 'Guter Preis'},
 {'Price': '€ 20.990,-', 'Label': 'Keine Angabe'},
 {'Price': '€ 11.999,-', 'Label': 'Sehr guter Preis'},
 {'Price': '€ 14.490,-', 'Label': 'Sehr guter Preis'},
 {'Price': '€ 18.470,-', 'Label': 'Fairer Preis'},
 {'Price': '€ 12.790,-', 'Label': 'Sehr guter Preis'},
 {'Price': '€ 12.590,-', 'Label': 'Guter Preis'},
 {'Price': '€ 13.590,-', 'Label': 'Fairer Preis'},
 {'Price': '€ 12.950,-', 'Label': 'Guter Preis'},
 {'Price': '€ 11.490,-', 'Label': 'Guter Preis'},
 {'Price': '€ 17.770,-', 'Label': 'Hoher Preis'},
 {'Price': '€ 16.430,-', 'Label': 'Sehr guter Preis'},
 {'Price': '€ 14.490,-', 'Label': 'Sehr guter Preis'}]

In [9]:
# Im nächsten Schritt wollen wir uns weitere verfügbare Informationen ansehen
# Hier können wir auf die technischen Details des Produkts zugreifen

url = "https://www.autoscout24.de/lst/volkswagen/golf/re_2015?source=listpage_seo_links"
response = requests.get(url)
soup = bs(response.content, 'html.parser')

container = soup.find("div", class_="VehicleDetailTable_container__mUUbY")

data_list = []
try:
    for item in container.find_all('span', {'class': 'VehicleDetailTable_item__koEV4'}):
        data_list.append(item.text)
except AttributeError:
    print("Data container not found or structure has changed.")

print(data_list)


['129.854 km', 'Schaltgetriebe', '01/2015', 'Diesel', '110 kW (150 PS)', '110 g/km (komb.) | 4,2 l/100 km (komb.)']


In [18]:
url= ("https://www.autoscout24.de/lst/volkswagen/golf?atype=C&cy=D&damaged_listing=exclude&desc=0&fregfrom=2020&ocs_listing"
    "=include&powertype=kw&search_id=22myllw9q7d&sort=standard&ustate=N%2CU")
response = requests.get(url)
soup = bs(response.content, 'html.parser')
container = soup.find_all("div", class_="ListItem_header__uPzec ListItem_header_new_design__hPPNh")

for c in container:
    t1 = c.find("h2").text.strip()
    t2 = c.find('span', class_="ListItem_version__jNjur").text if c.find('span', class_="ListItem_version__jNjur") is not None else ""
    t3 = c.find('span', class_="ListItem_subtitle__eY660").text if c.find('span', class_="ListItem_subtitle__eY660") is not None else ""

    print(t1, "\n", t2, "\n", t3, "\n\n\n")
    
    print(t1,"\n",t2,"\n", t3,"\n\n\n")

Volkswagen Golf VIII Lim. GTE ACC KAMERA SITZH PANO HEAD UP REG... 
 VIII Lim. GTE ACC KAMERA SITZH PANO HEAD UP REG... 
  



Volkswagen Golf VIII Lim. GTE ACC KAMERA SITZH PANO HEAD UP REG... 
 VIII Lim. GTE ACC KAMERA SITZH PANO HEAD UP REG... 
  



Volkswagen Golf VIII 1.5 TSI ACT United°LED°Virtual°Navi°SH 
 VIII 1.5 TSI ACT United°LED°Virtual°Navi°SH 
 Navigationssystem, 3-Zonen-Klimaautomatik, LED-Scheinwerfer, Totwinkel-Assistent, Volldigitales Kombiinstrument, Abstandstempomat, Induktionsladen für Smartphones, Sitzheizung 



Volkswagen Golf VIII 1.5 TSI ACT United°LED°Virtual°Navi°SH 
 VIII 1.5 TSI ACT United°LED°Virtual°Navi°SH 
 Navigationssystem, 3-Zonen-Klimaautomatik, LED-Scheinwerfer, Totwinkel-Assistent, Volldigitales Kombiinstrument, Abstandstempomat, Induktionsladen für Smartphones, Sitzheizung 



Volkswagen Golf VIII 1.5 TSI STYLE 5JG+IQ+NAV+ACC+PANO+AHK 
 VIII 1.5 TSI STYLE 5JG+IQ+NAV+ACC+PANO+AHK 
 LED-Scheinwerfer, Sportsitze, Anhängerkupplung, Klimaautomatik, 

## <a class="my-anchor2" id="ed"> Erstellung der Daten </a>

In [101]:
# Es werden Daten entsprechend der mit js erstellten Situation auf der Website generiert.
# Wir werden einen Scrape-Code schreiben, der den Status dieser Daten in der URL berücksichtigt. Wir haben dafür einige Listen erstellt.

cars = {'Mercedes-Benz': ['a-180', 'a-200','b-180','c180','c-200','c-220','cla-200','e-200','e-220'],
'Volkswagen': ['golf','golf-gti','golf-variant','passat','passat-variant','tiguan','touran','caddy','t-roc','polo'],
 'BMW': [116, 118, 218, 318, 320, 330, 420, 520, 530, 'x1', 'x2', 'x3', 'x5'],
 'Audi': ['a1','a3','a4','a4-allroad','a5','a6','a6-allroad','a7','a8','q7', 'q5','q3'],
 'Ford': ['ecosport','edge','focus','fiesta','kuga','c-max','mondeo','galaxy','s-max'],
 'Opel': ['agila','antara','astra','corsa','insignia','mokka','zafira','adam','grandland','grandland-x','crossland','crossland-x'],
 'Skoda': ['octavia', 'fabia', 'superb', 'kodiaq', 'karoq', 'kamiq', 'yeti']}

color_list = ["bc_beige","bc_blau","bc_braun","bc_bronze","bc_gelb","bc_grau","bc_grün",
            "bc_rot","bc_schwarz","bc_silber","bc_violet","bc_weiss","bc_orange", "bc_gold"]

schadstoff= [4, 5, 6, 7, 8, 9]    
# 4=4 /  5=5 / 6 = 6/ 7=6c /  8 = 6d, 9=6d-Temp    

new_liste = []

for br, model in cars.items():
    for md in model:
        for farbe in color_list:
            for sch in schadstoff:
                time.sleep(5)
                page_num = 1
                while True:
                    try:
                        response = requests.get(f"https://www.autoscout24.de/lst/{br}/{md}/{farbe}?atype=C&body=1%2C2%2C3%2C4%2C5%2C6&cy=D&"
                                                "damaged_listing=exclude&desc=0&emclass={sch}&fregfrom=2015&ocs_listing=include&page={page_num}"
                                                "&powertype=kw&search_id=28warkcbt2r&sort=standard&source=listpage_pagination&ustate=N%2CU")
                        soup = bs(response.content, 'html.parser')
                        container = soup.find_all("div", class_="ListItem_wrapper__J_a_C")
                        if not container:
                            break
                        for c in container:
                            data_list = []
                            for item in c.find_all('span', {'class': 'VehicleDetailTableNewDesign_item__5LQHk'}):
                                data_list.append(item.text)
                            price_element = c.find("p", class_="Price_price__WZayw PriceAndSeals_current_price__XscDn")
                            if price_element is not None:
                                price = price_element.text
                            lb_element = c.find("div", class_="scr-price-label PriceAndSeals_price_info___KqSg")
                            if lb_element is not None:
                                lb = lb_element.text
                            title_element = c.find("h2")
                            if title_element is not None:
                                title = title_element.text.strip()

                            new_liste.append({'Brand': br, 
                                            'Model': md, 
                                            'title':title,
                                            'Features': data_list, 
                                            'Price': price, 
                                            'Label': lb,
                                            'Color': farbe,
                                            'schadstoff': sch})
                    except requests.exceptions.RequestException as e:
                        print(f"Hata oluştu: {e}")  # Hata mesajını yazdır
                        print(response.status_code)
                        break  # Sonraki model ve yıla geç
                    except Exception as e:
                        print(f"Beklenmeyen bir hata oluştu: {e}")  # Hata mesajını yazdır
                        break  # Sonraki model ve yıla geç
                    page_num += 1   

# Now our data is ready, let's save this data as csv.
best = pd.DataFrame(new_liste)
best.to_csv("cars.csv", index=False)
best.info()

Hata oluştu: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))
200
Hata oluştu: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))
200
Hata oluştu: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))
200
Hata oluştu: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))
200
Hata oluştu: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))
200
Hata oluştu: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))
200
Hata oluştu: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbin

In [13]:
import string

# Es werden Daten entsprechend der mit js erstellten Situation auf der Website generiert. 
# Wir werden einen Scrape-Code schreiben, der den Status dieser Daten in der URL berücksichtigt. Wir haben dafür einige Listen erstellt.
cars = {'Mercedes-Benz': ['a-180', 'a-200','b-180','c-180','c-200','c-220','cla-200','e-200','e-220'],
'Volkswagen': ['golf','golf-gti','golf-variant','passat','passat-variant','tiguan','touran','caddy','t-roc','polo'],
 'BMW': [116, 118, 218, 318, 320, 330, 420, 520, 530, 'x1', 'x2', 'x3', 'x5'],
 'Audi': ['a1','a3','a4','a4-allroad','a5','a6','a6-allroad','a7','a8','q7', 'q5','q3'],
 'Ford': ['ecosport','edge','focus','fiesta','kuga','c-max','mondeo','galaxy','s-max'],
 'Opel': ['agila','antara','astra','corsa','insignia','mokka','zafira','adam','grandland','grandland-x','crossland','crossland-x'],
 'Skoda': ['octavia', 'fabia', 'superb', 'kodiaq', 'karoq', 'kamiq', 'yeti']}

color_list = ["bc_beige","bc_blau","bc_braun","bc_bronze","bc_gelb","bc_grau","bc_gr",
            "bc_rot","bc_schwarz","bc_silber","bc_violet","bc_wei","bc_orange", "bc_gold"]

year_ = [i for i in range(2015,2023,1)]

new_liste = []

for br, model in cars.items():
    current_datetime = datetime.now()
    print("Scrape yapılan Marka: ", br, "time: ", current_datetime)
    for md in model:
        for year in year_:
            for farbe in color_list:
                time.sleep(5)
                page_num = 1
                while True:
                    try:
                        # Die Response-Seite wird mit BeautifulSoup geparst, um die erforderlichen Informationen zu extrahieren
                        response = requests.get(f"https://www.autoscout24.de/lst/{br}/{md}/{farbe}?atype=C&body=1%2C2%2C3%2C4%2C5%2C6&"
                                                "cy=D&damaged_listing=exclude&desc=0&fregfrom={year}&fregto={year}&ocs_listing=include&"
                                                "page={page_num}&powertype=kw&search_id=28warkcbt2r&sort=standard&source=listpage_pagination&ustate=N%2CU")
                        soup = bs(response.content, 'html.parser')
                        container = soup.find_all("div", class_="ListItem_wrapper__J_a_C")
                        if not container:
                            # Wenn keine Angebote auf der Seite gefunden werden, wird die Schleife beendet
                            break
                        # Wir prüfen, ob das von der URL abgerufene Ergebnis ein Angebot enthält.
                        # Wenn es mindestens ein Angebot gibt, fahren wir fort, andernfalls wird die Schleife beendet
                        angebot_text = soup.find('div', class_='ListHeader_top__jY34N')
                        if int(angebot_text.text.split()[0].replace(".", "")) > 1:
                            # Durchlaufen aller Angebote (Container), um die erforderlichen Daten zu extrahieren
                            for c in container:
                                data_list = []
                                # Merkmale jedes Angebots werden gesammelt und in einer Liste gespeichert
                                for item in c.find_all('span', {'class':'VehicleDetailTable_item__koEV4'}):
                                    data_list.append(item.text)

                                # Der Preis jedes Angebots wird extrahiert und Sonderzeichen entfernt, um einen numerischen Wert zu erhalten
                                price_element = c.find("p", class_="Price_price__WZayw PriceAndSeals_current_price__XscDn")
                                if price_element is not None:
                                    price = price_element.text.translate(str.maketrans("", "", "€-,."))

                                # Die Beschriftung jedes Angebots wird extrahiert
                                lb_element = c.find("div", class_="scr-price-label PriceAndSeals_price_info___KqSg")
                                if lb_element is not None:
                                    lb = lb_element.text

                                # Der Titel (Name) jedes Angebots wird extrahiert und Leerzeichen werden entfernt    
                                title_element = c.find("h2")
                                if title_element is not None:
                                    title = title_element.text.strip()
                                    
                                # Alle extrahierten Informationen des Angebots werden der Liste 'new_liste' hinzugefügt
                                new_liste.append({'Brand': br, 
                                                'Model': md, 
                                                'title':title,
                                                'Features': data_list, 
                                                'Price': price, 
                                                'Label': lb,
                                                'Color': farbe})
                        else:
                            break
                    except requests.exceptions.RequestException as e:
                        print(f"Error: {e}")  
                        print(response.status_code)
                        break  # Wechsel zum nächsten Modell
                    except Exception as e:
                        print(f"Unexpected Error: {e}")  
                        break  # Wechsel zum nächsten Modell und Jahr
                    page_num += 1   

# Jetzt sind unsere Daten fertig, speichern wir sie als csv.
best = pd.DataFrame(new_liste)
best.to_csv("cars_new.csv", index=False)
best.info()

Scrape yapılan Marka:  Mercedes-Benz time:  2023-05-18 21:01:48.034331
Error: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))
200
Error: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))
200
Scrape yapılan Marka:  Volkswagen time:  2023-05-18 23:11:55.227540
Scrape yapılan Marka:  BMW time:  2023-05-19 01:38:55.548710
Error: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))
200
Error: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))
200
Error: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))
200
Scrape yapılan Marka:  Audi time:  2023-05-19 04:26:44.209790
Error: (

* [Home](#content)